In [1]:
include("datasetloader.jl")
include("train.jl")
include("trainutils.jl")
Threads.nthreads()

8

In [8]:
using BSON
using SampledVectors
ckpfn = "trainlogs/model.bson"
if isfile(ckpfn)
    ckp = BSON.load(ckpfn)
    M = ckp[:M]
    train_loss = ckp[:train_loss]
    default_q_list = ckp[:default_q_list]
    val_loss = ckp[:val_loss]
    val_accuracy = ckp[:val_accuracy]
else
    const PACKAGE_PATH = "."
    M = Model(joinpath(PACKAGE_PATH, "profiles"))
    # for Q in M.Qs
    #     for(k,v) in zip(keys(Q), shuffle(values(Q)|>collect))
    #         Q[k] = v
    #     end
    # end
end


Model(["ara", "bel", "ben", "bul", "cat", "ces", "dan", "deu", "ell", "eng"  …  "spa", "swa", "swe", "tat", "tgl", "tur", "ukr", "vie", "yid", "zho"], Dict("epo" => 11, "jpn" => 24, "kor" => 26, "ina" => 21, "ron" => 38, "deu" => 8, "nds" => 33, "fin" => 13, "kab" => 25, "bel" => 2…), Dict{Vector{UInt8}, Float32}[Dict([0xa7, 0xd9, 0x84, 0xd9, 0x82, 0xd8, 0xb7] => -11.586428, [0xd8, 0xac, 0xd9, 0x85, 0xd8, 0xb9, 0xd9] => -12.631353, [0x84, 0xd9, 0x83, 0x20, 0xd8, 0xa3, 0xd9] => -12.094713, [0x84, 0xd8, 0xb5, 0x20, 0xd8] => -12.442206, [0x83, 0xd9, 0x85, 0xd8, 0xa7] => -10.856149, [0x20, 0xd8, 0xab, 0xd8, 0xa7, 0xd9, 0x86] => -12.492708, [0xd8, 0xb1, 0xd8, 0xa9, 0x20, 0xd8, 0xa8] => -11.708431, [0x81, 0xd9, 0x8a, 0xd9, 0x87, 0xd8] => -11.517032, [0xa7, 0x20, 0xd8, 0xb7] => -12.00716, [0xd8, 0xb4, 0xd8, 0xb1, 0xd8, 0xb7] => -11.736457…), Dict([0xb0, 0xd1, 0x86, 0xd1, 0x8b, 0xd1, 0x8f] => -10.588074, [0xd0, 0xbf, 0xd0, 0xbb, 0xd0, 0xb0, 0xd1] => -11.378538, [0x91, 0xd0, 0xb4, 0xd1] => -11.

In [3]:
# using BSON
# using SampledVectors
# const PACKAGE_PATH = "."
# ckp = BSON.load()
# M = ckp[:M]
# train_loss = ckp[:train_loss]
# default_q_list = ckp[:default_q_list]
# val_loss = ckp[:val_loss]
# val_accuracy = ckp[:val_accuracy]

In [4]:
WDs = [WikiDataSet("corpus/wikipedia/test", langs=[l]) for l in M.languages]
TDs = [TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=[l]) for l in M.languages]
datasets = [WDs; TDs]
val_set = BatchedLoader(WeightedLoader(datasets, weights=log.(length.(datasets))), 2)
val_set_b10 = BatchedLoader(WeightedLoader(datasets, weights=log.(length.(datasets))), 10)

BatchedLoader(WeightedLoader(Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}[Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}(Base.Iterators.Cycle{RandomLoader}(RandomLoader(["آمونبرغ  هي بلدة، مدينة و بلدة ألمانية تقع في ألمانيا في Marburg-Biedenkopf. بحاجة لمصدر \nمدن شتولبن، Château-Garnier، تورو سول تراسيمينو و Tragwein هي مدن توأمة لـآمونبرغ.\nهذه بذرة مقالة عن موقع جغرافي في هسن بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آني بيدويل \u200f هي سفرجات أمريكية، ولدت في 1839، وتوفيت في 1918. 2  3  4 \nهذه بذرة مقالة عن عالم نبات أمريكي بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آنا هيوم \u200f ؛ مترجمة، كاتِبة وشاعرة بريطانية. 1  2  3 \nهذه بذرة مقالة عن  شاعر بريطاني بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آني لافلور \u200f ؛ كاتِبة وشاعرة كندية. 1  2 \nهذه بذرة مقالة عن  شاعر كندي بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آيت يعقوب هو دُوَّار يقع

In [5]:
WDs = [WikiDataSet("corpus/wikipedia/train", langs=[l]) for l in M.languages]
TDs = [TatoebaDataset("corpus/tatoeba", "tatoeba_train.txt", langs=[l]) for l in M.languages]
datasets = [WDs; TDs]
train_set = BatchedLoader(WeightedLoader(datasets, weights=log.(length.(datasets))), 2)

BatchedLoader(WeightedLoader(Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}[Base.Iterators.Stateful{Base.Iterators.Cycle{RandomLoader}, Union{Nothing, Tuple{Any, Int64}}, Int64}(Base.Iterators.Cycle{RandomLoader}(RandomLoader(["آب\u200cبادامو  هي قرية تقع في قسم جايدشت الريفي  في إيران. 1  يقدر عدد سكانها بـ 0 نسمة بحسب إحصاء 2016. 2 \nهذه بذرة مقالة عن موقع جغرافي في محافظة فارس بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "آتوس سوانتس  هو مبارز بالسيف برازيلي، مثّل بلاده في الألعاب الأولمبية الصيفية 2012. 1 \nآتوس سوانتس على موقع FIE \nهذه بذرة مقالة عن مبارز سيف شيش برازيلي بحاجة للتوسيع. فضلًا شارك في تحريرها." => "ara", "تعديل - تعديل مصدري - تعديل ويكي بياناتآخر أيام الأرض فيلم صوتي درامي من تأليف وإخراج خالد المهدي وبطولة حنان ترك وخالد صالح وهاني عادل وخالد النبوي وعمرو القاضي. 1  2  3 \nتدور أحداث الفيلم عن التصور الإسلامي لنهاية العالم، حيث يروي قصة 40 يوما من عمر البشرية تحفل بالأحداث المصيرية وفي مقدمتها المعركة الف

In [16]:
get_accuracy(M, first(val_set_b10))

1.0

In [44]:
# @time data = first(val_set);
# @time get_loss(M, first(val_set))
# @time l, grad = loss_and_grad(M, data, 5);
# # @time step!(M, grad)
# get_loss(M, first(val_set))


  0.013364 seconds (175 allocations: 11.383 KiB)


  1.488831 seconds (14.02 M allocations: 366.403 MiB, 32.02% gc time)


(0.848577685325165, (Qs = Dict{Any, Any}[Dict(UInt8[0x87, 0xd8] => -6.219567f-6, UInt8[0xa7, 0x20] => -6.213986f-6, UInt8[0x89, 0x20] => 2.3854392f-9, UInt8[0xd8, 0xb3, 0xd9] => -6.219668f-6, UInt8[0xd8, 0xaf] => -6.211142f-6, UInt8[0x20, 0xd9, 0x85, 0xd9] => -9.329937f-6, UInt8[0x85, 0x20] => -6.2187823f-6, UInt8[0xb1, 0xd9] => -2.175681f-5, UInt8[0xd9, 0x8a, 0x20, 0xd9] => -6.2201275f-6, UInt8[0x20] => -0.00010613138f0…), Dict(UInt8[0xb2] => 0.00029645365f0, UInt8[0xbd, 0xd0] => 0.0005926175f0, UInt8[0xd0, 0xb0, 0x20] => 0.0005927524f0, UInt8[0xb5] => 3.9111034f-9, UInt8[0xb0, 0xd0] => 0.000590646f0, UInt8[0x8f] => 0.00029632065f0, UInt8[0x20, 0xd0, 0xbd, 0xd0] => 0.00029686242f0, UInt8[0xbd, 0xd0, 0xb0] => 0.00059336f0, UInt8[0x20, 0xd1, 0x8f] => 0.000296917f0, UInt8[0xbb, 0xd0] => 0.00029673427f0…), Dict(UInt8[0xa6] => 3.6163928f-7, UInt8[0xa4] => 1.2037481f-5, UInt8[0xac] => 2.5584944f-7, UInt8[0xa4, 0x20, 0xe0] => 1.2660313f-7, UInt8[0x95] => 1.1382826f-6, UInt8[0xa8, 0x20] => 1.

In [75]:
# plot_series("train_loss" => train_loss, "val_loss" => val_loss, title="loss")

In [10]:
# include("train.jl")
# M = Model(joinpath(PACKAGE_PATH, "ngrams"))
train_loss = SampledVector{Float64}(4096*4)
val_loss = SampledVector{Tuple{Int, Float64}}(4096*4)
val_accuracy = SampledVector{Tuple{Int, Float64}}(4096*4)
default_q_list = SampledVector{Float64}(4096*4)
cutoff_list = SampledVector{Tuple{Int, Float64}}(4096*4)

0-element SampledVector{Tuple{Int64, Float64}, Int64, Int64, Int64, Int64}

In [12]:
# ckp = BSON.load("model.bson")
# M = ckp[:M]
# train_loss = ckp[:train_loss]
# default_q_list = ckp[:default_q_list];

In [16]:
mkpath("trainlogs")
for i in 1:8400*2*1
    l, grad = loss_and_grad(M, first(train_set), rand([1:M.ngram; 3:M.ngram; 3:min(5, M.ngram)]))
    step!(M, grad, 1f0)
    push!(train_loss, l)
    push!(default_q_list, M.default_q)
    iters = length(train_loss)
    if iters % 10 == 0
        l_val = get_loss(M, first(val_set))
        push!(val_loss, (length(train_loss), l_val))
        acc_val = get_accuracy(M, first(val_set_b10))
        push!(val_accuracy, (length(train_loss), acc_val))
    end
    if iters % 50 == 0
        plot_series("train_loss" => train_loss, "val_loss" => val_loss, title="loss", path="trainlogs")
        plot_series("val_accuracy" => val_accuracy, title="accuracy", path="trainlogs")
        plot_series("default_q" => default_q_list, path="trainlogs")
    end
    if iters % 100 == 0
        # random_cutoff(M)
        push!(cfm, (length(train_loss), sum(M.cutoff_list)/length(M.cutoff_list)))
        plot_series("cutoff" => cfm, path="trainlogs")
    end
    if iters % 2000 == 0
        bson("trainlogs/model.bson", M=M, train_loss=train_loss, val_loss=val_loss, 
        default_q_list=default_q_list, val_accuracy=val_accuracy, cutoff_list=cutoff_list)
    end
end
bson("trainlogs/model.bson", M=M, train_loss=train_loss, val_loss=val_loss, 
default_q_list=default_q_list, val_accuracy=val_accuracy, cutoff_list=cutoff_list)
length(train_loss)


151259

In [33]:
l, grad = loss_and_grad(M, first(train_set), rand([1:M.ngram; 3:M.ngram; 3:min(5, M.ngram)]))


(4.311733326713481, (Qs = Dict{Any, Any}[Dict(UInt8[0xb0, 0xd0, 0xb3] => -4.9986494f-7, UInt8[0xd1, 0x85] => -2.4983708f-6, UInt8[0x88, 0xd0, 0xb5] => -7.497631f-7, UInt8[0xbd, 0xd0] => -6.470823f-5, UInt8[0xba, 0xd0, 0xbb] => -2.4993824f-7, UInt8[0xd0, 0xb8, 0xd1] => -2.6941238f-5, UInt8[0xb0, 0xd0] => -4.458852f-5, UInt8[0xb2, 0xd0, 0xbe] => -1.849007f-5, UInt8[0xd1, 0x88, 0x20] => -2.4991292f-7, UInt8[0xb8, 0x20] => -3.9400296f-5…), Dict(UInt8[0x20, 0x63] => -0.008029935f0, UInt8[0x76, 0x69] => 1.4828688f-5, UInt8[0x69, 0x20] => 1.4819421f-5, UInt8[0x65, 0x72] => -0.008023357f0, UInt8[0x72] => -0.007956818f0, UInt8[0x65, 0x20] => -0.015976109f0, UInt8[0x20] => -0.02661635f0, UInt8[0x20, 0x63, 0x6c] => -0.008040617f0, UInt8[0x64, 0x20, 0x68] => -0.008039994f0, UInt8[0x78, 0x20] => 1.4831886f-5…), Dict(UInt8[0xb0, 0xd0, 0xb3] => -5.492826f-7, UInt8[0xd1, 0x85] => -2.7455822f-6, UInt8[0x88, 0xd0, 0xb5] => -8.238988f-7, UInt8[0xbd, 0xd0] => -7.12007f-5, UInt8[0x98, 0xd1] => -3.2951134f-

In [50]:
# @time random_cutoff(M)

  0.685796 seconds (5.84 M allocations: 282.813 MiB, 8.52% gc time)


6